In [1]:
from keras.models  import load_model
import keras.backend as K
from sklearn.preprocessing import normalize
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
from colors import color
import pandas as pd
import numpy as np

aspects = [str(i) for i in range(2)] #app, aroma, palate,taste

Using TensorFlow backend.


In [3]:
vec = cPickle.load(open('../../beer_data/beer_vec_ds_2000.p', 'rb'))

In [4]:
X = vec.X

In [2]:
model = load_model('../store/weights/beer/ThuNov1614:08:102017/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b})

In [5]:
model.get_layer('pred_0').get_layer('pool_0').summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 305)           0                                            
____________________________________________________________________________________________________
embedding (Embedding)            (None, 305, 300)      600600      input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_4 (Conv1D)                (None, 305, 256)      230656      embedding[0][0]                  
____________________________________________________________________________________________________
conv1d_5 (Conv1D)                (None, 305, 256)      230656      embedding[0][0]                  
___________________________________________________________________________________________

In [6]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs, 
                          [model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).layers[3].output])

In [7]:
filters_pico = {}
sample_length = 1000
sample = np.random.choice(X.shape[0], sample_length, replace=False)
i, batch_size = 0, 64
for aspect in aspects :
    filters_pico[aspect] = model_c1[aspect]([X[sample]])[0]
    
# i, bs = 0, 64
# vecs = {k:[] for k in aspects}
# while i*bs < len(X):
#     for aspect in aspects :
#         samples = X[i*bs:(i+1)*bs]
#         result = model_c1[aspect]([samples])[0]
#         vecs[aspect].append(result)
#     i += 1
#     print i
    
# for aspect in aspects :
#     result = np.concatenate(vecs[aspect], axis=0)
#     filters_pico[aspect] = result

In [8]:
nb_act = {k:(v > 0).sum()/float(len(X[sample])) for k,v in filters_pico.items()}
for key in sorted(nb_act.keys()) :
    print key,nb_act[key]

0 323.524
1 777.077


In [12]:
activated_words = {}
for aspect in aspects :
    filters = filters_pico[aspect]
    word_activations = np.zeros((vec.vocab_size, filters.shape[2]))
    for i in range(filters.shape[2]) :
        filter_i = filters[:,:,i]
        word2filter = np.dstack((X[sample], filter_i))
        filter_w2f = word2filter[word2filter[:,:,1] > 0.0]
        for word in filter_w2f :
            word_activations[int(word[0]), i] = word[1]
    activated_words[aspect] = word_activations

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
top_words_df = pd.DataFrame()
top_words_size = 40
for aspect in aspects :
    popwords = activated_words[aspect].sum(axis=1)
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    top_words_df[aspect] = top_words
top_words_df

In [ ]:
summed = {}
for aspect in aspects :
    summed[aspect] = filters_pico[aspect].sum(axis=-1)

In [9]:
X_words = map(lambda s : [vec.idx2word[w] for w in s], X[sample])

In [ ]:
zipped = zip(X_words, summed['0'], summed['1'], summed['2'], summed['3'])
total_vals = []
for x,a0,a1,a2,a3 in zipped :
    total_vals.append(zip(x,a0,a1,a2,a3))
    
for abst in total_vals :
    for sent in abst :
         if sent[0] != '[0]' :
            if sent[1] > 0 and sent[2] > 0 and sent[3] > 0:
                print "\033[1;33;48m "+sent[0],
            elif sent[1] > 0 :
                print "\033[1;31;48m "+sent[0], #r app
            elif sent[2] > 0 :
                print "\033[1;32;48m "+sent[0], #g aro
            elif sent[3] > 0 :
                print "\033[1;34;48m "+sent[0], #b pal
            elif sent[4] > 0 :
                print "\033[1;36;48m "+sent[0], #c taste
            else :
                print "\033[0;30;48m "+sent[0],
        
    print ''

In [ ]:
get_ipython().system('cp "Filter Viz Single.ipynb" '+results_folder)

In [16]:
gate0 = filters_pico['0'][20, :, :].sum(axis=-1)#/len(X[sample])
gate1 = filters_pico['1'][20, :, :].sum(axis=-1)#/len(X[sample])
gate = normalize([gate0, gate1])
zip(gate0, gate1, X_words[20])

[(0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),
 (0.0, 0.0, '[0]'),


In [12]:
import pandas as pd
ds = pd.read_csv('../../beer_data/beer_ds.csv')


In [6]:
aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])
idxs = ds.index
sample_length = 10000
idxs = np.random.choice(idxs, sample_length, replace=False)
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [13]:
from sklearn.metrics import roc_auc_score
aucs = [0] * sample_length
for i in range(sample_length) :
    aucs[i] = roc_auc_score(H['1'][i], H['0'][i])
print np.mean(aucs)

0.868109145028
